# Librerías

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os

Listado del total de los datos.

In [2]:
os.listdir('../data/')

['product_blocks.csv',
 'products.csv',
 'stock_day_71_stock_replenishment_days_72-77.csv',
 'positions_day_71.csv',
 'positions_day_0_day_63.csv',
 'stock_and_sales_day_0_day_63.csv']

#### Product_blocks

In [3]:
product_blocks = pd.read_csv('../data/product_blocks.csv')

In [4]:
product_blocks.head()

,product_id,block_id
0,612967398,0
1,296892108,0
2,139541214,0
3,963923934,0
4,938230141,0


In [5]:
product_blocks.block_id.value_counts().head()

2506    10
1717    10
2666    10
962     10
407     10
Name: block_id, dtype: int64

#### Products

In [6]:
products = pd.read_csv('../data/products.csv')
products.head()

,product_id,family_id,subfamily_id,price
0,151926,396066037,335531561,25.95
1,213413,552529755,11509337,19.95
2,310130,679611953,533441312,12.95
3,373883,775013441,62518413,17.95
4,455200,998145072,490222156,29.95


#### Stock_day_71_stock_replenishment_days_72-77

In [7]:
day_71_77 = pd.read_csv('../data/stock_day_71_stock_replenishment_days_72-77.csv')
day_71_77.head()

,product_id,color_id,size_id,stock_at_day_71,stock_replenishment_days_72,stock_replenishment_days_73,stock_replenishment_days_74,stock_replenishment_days_75,stock_replenishment_days_76,stock_replenishment_days_77
0,151926,909958217,54666687,73,0,0,2,0,0,1
1,151926,909958217,111455099,16,0,0,0,0,0,0
2,151926,909958217,242364222,111,0,1,-1,1,0,0
3,151926,909958217,513017386,47,0,0,0,1,1,1
4,151926,909958217,820445075,71,0,1,-2,2,0,0


In [8]:
len(day_71_77.product_id.value_counts())

8892

#### Positions_day_71-77

In [9]:
position_71 = pd.read_csv('../data/positions_day_71.csv')
position_71.head()

,date_number,product_id,category_id,position
0,71,65968023,2312968,6
1,71,70320877,2312968,24
2,71,70700940,2312968,22
3,71,76636979,2312968,1
4,71,83181798,2312968,0


In [10]:
position_71.date_number.value_counts()

71    20029
Name: date_number, dtype: int64

In [11]:
len(position_71.product_id.value_counts())

7728

#### Positions_day_0_day_63

In [12]:
position_0_63 = pd.read_csv('../data/positions_day_0_day_63.csv')
position_0_63.head()

,date_number,product_id,category_id,position
0,0,4450020,4461548,17
1,0,42147334,4461548,4
2,0,81131830,4461548,35
3,0,84035833,4461548,38
4,0,125252584,4461548,39


In [13]:
position_0_63.date_number.value_counts().head()

63    18688
61    18472
60    18429
62    18249
59    18244
Name: date_number, dtype: int64

#### Stock_and_sales_day_0_day_63

In [14]:
sales_0_63 = pd.read_csv('../data/stock_and_sales_day_0_day_63.csv')
sales_0_63.head()

,date_number,product_id,color_id,size_id,sales,stock
0,0,310130,78467031,7026791,1,93
1,0,310130,78467031,8015276,2,62
2,0,310130,78467031,452052101,1,18
3,0,310130,78467031,541898038,2,42
4,0,310130,78467031,558685710,3,113


# Generación de los conjuntos TRAIN / TEST

El primer objetivo de la competición es formar un conjunto de entrenamiento de modelos y otro de test.

Para comenzar, iremos mergeando los diferentes datasets hasta dar con un primer conjunto de datos inicial. Uniremos los datasets de ventas-stock y posicionamiento en la web para los días de 0 a 63

## TRAIN

In [75]:
df_0_63 = position_0_63.join(sales_0_63.set_index(['date_number', 'product_id']),
                             on=['date_number', 'product_id'],
                             how='right')
df_0_63.head()

,date_number,product_id,category_id,position,color_id,size_id,sales,stock
0,0,4450020,4461548.0,17.0,10926303,242364222,1,1
42,0,4450020,6649227.0,76.0,10926303,242364222,1,1
6058,0,4450020,563537811.0,48.0,10926303,242364222,1,1
7692,0,4450020,702067879.0,76.0,10926303,242364222,1,1
8482,0,4450020,751330438.0,2.0,10926303,242364222,1,1


In [76]:
print(sales_0_63.shape, position_0_63.shape, df_0_63.shape)

(2421061, 6) (969897, 4) (5607224, 8)


Nulls

In [77]:
df_0_63.isnull().sum().sum()

373884

In [82]:
nan_rows = df_0_63[df_0_63.isnull().T.any().T]
nan_rows[nan_rows.sales>0].head()

,date_number,product_id,category_id,position,color_id,size_id,sales,stock
969896,0,6339011,NaN,NaN,165718507,3000471,1,22
969896,0,6339011,NaN,NaN,165718507,558685710,1,25
969896,0,17705785,NaN,NaN,271590995,820445075,2,104
969896,0,18904572,NaN,NaN,431398116,820445075,1,1
969896,0,21302554,NaN,NaN,523743496,111455099,1,74


In [88]:
df_0_63_products = df_0_63.join(products.set_index(['product_id']), on=['product_id'], how='left')
df_0_63_products.head()

,date_number,product_id,category_id,position,color_id,size_id,sales,stock,family_id,subfamily_id,price
0,0,4450020,4461548.0,17.0,10926303,242364222,1,1,511255811,731664934,12.95
42,0,4450020,6649227.0,76.0,10926303,242364222,1,1,511255811,731664934,12.95
6058,0,4450020,563537811.0,48.0,10926303,242364222,1,1,511255811,731664934,12.95
7692,0,4450020,702067879.0,76.0,10926303,242364222,1,1,511255811,731664934,12.95
8482,0,4450020,751330438.0,2.0,10926303,242364222,1,1,511255811,731664934,12.95


In [89]:
nan_rows = df_0_63_products[df_0_63_products.isnull().T.any().T]
nan_rows[nan_rows.sales>0].head()

,date_number,product_id,category_id,position,color_id,size_id,sales,stock,family_id,subfamily_id,price
969896,0,6339011,NaN,NaN,165718507,3000471,1,22,998145072,309388913,59.95
969896,0,6339011,NaN,NaN,165718507,558685710,1,25,998145072,309388913,59.95
969896,0,17705785,NaN,NaN,271590995,820445075,2,104,775013441,478492088,25.95
969896,0,18904572,NaN,NaN,431398116,820445075,1,1,775013441,478492088,15.99
969896,0,21302554,NaN,NaN,523743496,111455099,1,74,744793598,330273563,29.95


In [90]:
print(df_0_63_products.shape, df_0_63.shape, products.shape)

(5607224, 11) (5607224, 8) (15238, 4)


In [111]:
df_0_63_products_blocks = df_0_63_products.join(product_blocks.set_index(['product_id']), on=['product_id'], how='left')
df_0_63_products_blocks.head(5)

,date_number,product_id,category_id,position,color_id,size_id,sales,stock,family_id,subfamily_id,price,block_id
0,0,4450020,4461548.0,17.0,10926303,242364222,1,1,511255811,731664934,12.95,1779
42,0,4450020,6649227.0,76.0,10926303,242364222,1,1,511255811,731664934,12.95,1779
6058,0,4450020,563537811.0,48.0,10926303,242364222,1,1,511255811,731664934,12.95,1779
7692,0,4450020,702067879.0,76.0,10926303,242364222,1,1,511255811,731664934,12.95,1779
8482,0,4450020,751330438.0,2.0,10926303,242364222,1,1,511255811,731664934,12.95,1779


#### Generación del train final

In [203]:
def DF_generator(df_in, tr=True):
    n_unique = df_in.groupby(['date_number', 'product_id']).nunique()
    max_df = df_in.groupby(['date_number', 'product_id']).max()
    sum_df = df_in.groupby(['date_number', 'product_id']).sum()
        
    sum_sales = sum_df['sales']
    sum_stock = sum_df['stock']
    count_category = n_unique['category_id']
    max_position = max_df['position']
    min_position = df_in.groupby(['date_number', 'product_id']).min()['position']
    count_color = n_unique['color_id']
    count_size = n_unique['size_id']
    count_family = max_df['family_id']
    count_subfamily = max_df['subfamily_id']
    price = max_df['price']
    block = max_df['block_id']
    
    df_out = pd.concat([block,sum_stock,count_family,count_subfamily,count_size,count_color,
                   max_position,min_position,count_category,price,sum_sales], axis=1)
    df_out.columns = ['stock', 'block_id', 'family_id', 'subfamily_id', 'size_id',
           'color_id', 'position_max', 'position_min', 'category_id', 'price', 'sales']
    
    return df_out

In [169]:
# n_unique = df_0_63_products_blocks.groupby(['date_number', 'product_id']).nunique()
# max_df = df_0_63_products_blocks.groupby(['date_number', 'product_id']).max()
# sum_df = df_0_63_products_blocks.groupby(['date_number', 'product_id']).sum()

# sum_sales = sum_df['sales']
# sum_stock = sum_df['stock']
# count_category = n_unique['category_id']
# max_position = max_df['position']
# min_position = df_0_63_products_blocks.groupby(['date_number', 'product_id']).min()['position']
# count_color = n_unique['color_id']
# count_size = n_unique['size_id']
# count_family = max_df['family_id']
# count_subfamily = max_df['subfamily_id']
# price = max_df['price']
# block = max_df['block_id']

# train = pd.concat([sum_stock,block,count_family,count_subfamily,count_size,count_color,
#                    max_position,min_position,count_category,price,sum_sales], axis=1)
# train.columns = ['stock', 'block_id', 'family_id', 'subfamily_id', 'size_id',
#        'color_id', 'position_max', 'position_min', 'category_id', 'price', 'sales']
# train.head(10)

In [184]:
train = DF_generator(df_0_63_products_blocks)
train.head(10)

stock  block_id  family_id  subfamily_id  size_id  \
date_number product_id                                                      
0           310130       1726      1383  679611953     533441312        7   
            1178388       592        60  732697347     691762817        4   
            1561460      1625      2373  396066037     520569701        5   
            1874414      1135      1686  744793598     811402796        6   
            2436420       779       245  768025921     665805124        5   
            2637932       768        57  429269907     778576411        6   
            2644529      1339      1332  831347344     373740397        8   
            3093333       123        13  511255811     194174548        2   
            3129413      1027      2168  953774311     486307114        6   
            3176725       546      1086  396066037     335531561        4   

                        color_id  position_max  position_min  category_id  \
date_number product_id                                                      
0           310130             1          80.0           3.0            3   
            1178388            1          19.0          19.0            1   
            1561460            1          38.0           3.0            3   
            1874414            1         190.0          12.0            6   
            2436420            1           NaN           NaN            0   
            2637932            1           NaN           NaN            0   
            2644529            1          26.0           4.0            3   
            3093333            1          70.0          70.0            1   
            3129413            2         181.0          14.0            4   
            3176725            1         131.0           9.0            3   

                        price  sales  
date_number product_id                
0           310130      12.95     33  
            1178388     49.95      0  
            1561460     29.95     21  
            1874414     25.95     24  
            2436420     25.95      0  
            2637932     79.95      0  
            2644529      7.95     18  
            3093333     35.95      4  
            3129413     39.99    148  
            3176725     29.95     27

#### Nulls

In [171]:
train.isnull().sum().sum()

86006

In [172]:
nan_rows = train[train.isnull().T.any().T]
nan_rows[nan_rows.sales>0].head()

stock  block_id  family_id  subfamily_id  size_id  \
date_number product_id                                                      
0           6339011      2272       158  998145072     309388913        7   
            17705785     2198       104  775013441     478492088        1   
            18904572      430         1  775013441     478492088        1   
            21302554     2028       596  744793598     330273563        5   
            23383691      216         1  877011792     723618309        1   

                        color_id  position_max  position_min  category_id  \
date_number product_id                                                      
0           6339011            1           NaN           NaN            0   
            17705785           1           NaN           NaN            0   
            18904572           1           NaN           NaN            0   
            21302554           2           NaN           NaN            0   
            23383691           1           NaN           NaN            0   

                        price  sales  
date_number product_id                
0           6339011     59.95      2  
            17705785    25.95      2  
            18904572    15.99      1  
            21302554    29.95      3  
            23383691    12.95      1

Con esto que hemos hecho tendríamos el conjunto de entrenamiento. Ahora tenemos q sacar los "diferentes" conjuntos de validación.



## TEST

Para comenzar separaremos el conjunto de "stock" para los diferentes dias (71, 72, 73, 74, 75, 76 y 77)

In [25]:
day_71_77.head()

,product_id,color_id,size_id,stock_at_day_71,stock_replenishment_days_72,stock_replenishment_days_73,stock_replenishment_days_74,stock_replenishment_days_75,stock_replenishment_days_76,stock_replenishment_days_77
0,151926,909958217,54666687,73,0,0,2,0,0,1
1,151926,909958217,111455099,16,0,0,0,0,0,0
2,151926,909958217,242364222,111,0,1,-1,1,0,0
3,151926,909958217,513017386,47,0,0,0,1,1,1
4,151926,909958217,820445075,71,0,1,-2,2,0,0


In [26]:
day_71_77.columns

Index(['product_id', 'color_id', 'size_id', 'stock_at_day_71',
       'stock_replenishment_days_72', 'stock_replenishment_days_73',
       'stock_replenishment_days_74', 'stock_replenishment_days_75',
       'stock_replenishment_days_76', 'stock_replenishment_days_77'],
      dtype='object')

In [27]:
stock_71 = day_71_77[['product_id', 'color_id', 'size_id']]
stock_71['stock'] = day_71_77['stock_at_day_71']
stock_71['date_number'] = 71
stock_71.head()

,product_id,color_id,size_id,stock,date_number
0,151926,909958217,54666687,73,71
1,151926,909958217,111455099,16,71
2,151926,909958217,242364222,111,71
3,151926,909958217,513017386,47,71
4,151926,909958217,820445075,71,71


In [28]:
stock_72 = day_71_77[['product_id', 'color_id', 'size_id']]
stock_72['stock'] = stock_71['stock'] + day_71_77['stock_replenishment_days_72']
stock_72['date_number'] = 72
stock_72.head()

,product_id,color_id,size_id,stock,date_number
0,151926,909958217,54666687,73,72
1,151926,909958217,111455099,16,72
2,151926,909958217,242364222,111,72
3,151926,909958217,513017386,47,72
4,151926,909958217,820445075,71,72


In [29]:
stock_73 = day_71_77[['product_id', 'color_id', 'size_id']]
stock_73['stock'] = stock_72['stock'] + day_71_77['stock_replenishment_days_73']
stock_73['date_number'] = 73
stock_73.head()

,product_id,color_id,size_id,stock,date_number
0,151926,909958217,54666687,73,73
1,151926,909958217,111455099,16,73
2,151926,909958217,242364222,112,73
3,151926,909958217,513017386,47,73
4,151926,909958217,820445075,72,73


In [30]:
stock_74 = day_71_77[['product_id', 'color_id', 'size_id']]
stock_74['stock'] = stock_73['stock'] + day_71_77['stock_replenishment_days_74']
stock_74['date_number'] = 74
stock_74.head()

,product_id,color_id,size_id,stock,date_number
0,151926,909958217,54666687,75,74
1,151926,909958217,111455099,16,74
2,151926,909958217,242364222,111,74
3,151926,909958217,513017386,47,74
4,151926,909958217,820445075,70,74


In [31]:
stock_75 = day_71_77[['product_id', 'color_id', 'size_id']]
stock_75['stock'] = stock_74['stock'] + day_71_77['stock_replenishment_days_75']
stock_75['date_number'] = 75
stock_75.head()

,product_id,color_id,size_id,stock,date_number
0,151926,909958217,54666687,75,75
1,151926,909958217,111455099,16,75
2,151926,909958217,242364222,112,75
3,151926,909958217,513017386,48,75
4,151926,909958217,820445075,72,75


In [32]:
stock_76 = day_71_77[['product_id', 'color_id', 'size_id']]
stock_76['stock'] = stock_75['stock'] + day_71_77['stock_replenishment_days_76']
stock_76['date_number'] = 76
stock_76.head()

,product_id,color_id,size_id,stock,date_number
0,151926,909958217,54666687,75,76
1,151926,909958217,111455099,16,76
2,151926,909958217,242364222,112,76
3,151926,909958217,513017386,49,76
4,151926,909958217,820445075,72,76


In [33]:
stock_77 = day_71_77[['product_id', 'color_id', 'size_id']]
stock_77['stock'] = stock_76['stock'] + day_71_77['stock_replenishment_days_77']
stock_77['date_number'] = 77
stock_77.head()

,product_id,color_id,size_id,stock,date_number
0,151926,909958217,54666687,76,77
1,151926,909958217,111455099,16,77
2,151926,909958217,242364222,112,77
3,151926,909958217,513017386,50,77
4,151926,909958217,820445075,72,77


Ahora podemos unir el dataset al completo

In [34]:
stock_71_77 = pd.concat([stock_71, stock_72 ,stock_73 ,stock_74 ,stock_75 ,stock_76 ,stock_77]).reset_index(drop=True)

In [35]:
stock_71_77.columns

Index(['product_id', 'color_id', 'size_id', 'stock', 'date_number'], dtype='object')

In [36]:
stock_71_77 = stock_71_77[['date_number', 'product_id', 'color_id', 'size_id', 'stock']]

In [37]:
stock_71_77.head()

,date_number,product_id,color_id,size_id,stock
0,71,151926,909958217,54666687,73
1,71,151926,909958217,111455099,16
2,71,151926,909958217,242364222,111
3,71,151926,909958217,513017386,47
4,71,151926,909958217,820445075,71


In [38]:
stock_71_77.shape

(358876, 5)

In [163]:
products.shape

(15238, 4)

In [39]:
df_71_77_products = stock_71_77.join(products.set_index(['product_id']), on=['product_id'], how='inner')
df_71_77_products.head()

,date_number,product_id,color_id,size_id,stock,family_id,subfamily_id,price
0,71,151926,909958217,54666687,73,396066037,335531561,25.95
1,71,151926,909958217,111455099,16,396066037,335531561,25.95
2,71,151926,909958217,242364222,111,396066037,335531561,25.95
3,71,151926,909958217,513017386,47,396066037,335531561,25.95
4,71,151926,909958217,820445075,71,396066037,335531561,25.95


In [40]:
df_71_77_products.shape

(358876, 8)

In [164]:
product_blocks.shape

(15238, 2)

In [41]:
df_71_77_products_blocks = df_71_77_products.join(product_blocks.set_index(['product_id']), on=['product_id'], how='inner')
df_71_77_products_blocks.head()

,date_number,product_id,color_id,size_id,stock,family_id,subfamily_id,price,block_id
0,71,151926,909958217,54666687,73,396066037,335531561,25.95,1969
1,71,151926,909958217,111455099,16,396066037,335531561,25.95,1969
2,71,151926,909958217,242364222,111,396066037,335531561,25.95,1969
3,71,151926,909958217,513017386,47,396066037,335531561,25.95,1969
4,71,151926,909958217,820445075,71,396066037,335531561,25.95,1969


In [165]:
df_71_77_products_blocks.shape, position_71.shape

((358876, 9), (20029, 4))

In [69]:
df_71_77_products_blocks_positions = df_71_77_products_blocks.join(position_71.set_index(['date_number', 'product_id']),
                             on=['date_number', 'product_id'],
                             how='left')
df_71_77_products_blocks_positions.head()

,date_number,product_id,color_id,size_id,stock,family_id,subfamily_id,price,block_id,category_id,position
0,71,151926,909958217,54666687,73,396066037,335531561,25.95,1969,447647762.0,5.0
0,71,151926,909958217,54666687,73,396066037,335531561,25.95,1969,653552053.0,22.0
1,71,151926,909958217,111455099,16,396066037,335531561,25.95,1969,447647762.0,5.0
1,71,151926,909958217,111455099,16,396066037,335531561,25.95,1969,653552053.0,22.0
2,71,151926,909958217,242364222,111,396066037,335531561,25.95,1969,447647762.0,5.0


In [70]:
df_71_77_products_blocks_positions.shape

(430830, 11)

In [73]:
nan_rows = df_71_77_products_blocks_positions[df_71_77_products_blocks_positions.isnull().T.any().T]
nan_rows.head()

,date_number,product_id,color_id,size_id,stock,family_id,subfamily_id,price,block_id,category_id,position
51268,72,151926,909958217,54666687,73,396066037,335531561,25.95,1969,NaN,NaN
51269,72,151926,909958217,111455099,16,396066037,335531561,25.95,1969,NaN,NaN
51270,72,151926,909958217,242364222,111,396066037,335531561,25.95,1969,NaN,NaN
51271,72,151926,909958217,513017386,47,396066037,335531561,25.95,1969,NaN,NaN
51272,72,151926,909958217,820445075,71,396066037,335531561,25.95,1969,NaN,NaN


In [205]:
df_71_77_products_blocks_positions['sales'] = -1

In [206]:
df_71_77_products_blocks_positions.head()

,date_number,product_id,color_id,size_id,stock,family_id,subfamily_id,price,block_id,category_id,position,sales
0,71,151926,909958217,54666687,73,396066037,335531561,25.95,1969,447647762.0,5.0,-1
0,71,151926,909958217,54666687,73,396066037,335531561,25.95,1969,653552053.0,22.0,-1
1,71,151926,909958217,111455099,16,396066037,335531561,25.95,1969,447647762.0,5.0,-1
1,71,151926,909958217,111455099,16,396066037,335531561,25.95,1969,653552053.0,22.0,-1
2,71,151926,909958217,242364222,111,396066037,335531561,25.95,1969,447647762.0,5.0,-1


In [208]:
test = DF_generator(df_in=df_71_77_products_blocks_positions)
test['sales'] = -1
test.head(10)

stock  block_id  family_id  subfamily_id  size_id  \
date_number product_id                                                      
71          151926       1969       636  396066037     335531561        5   
            213413       1648      1190  552529755      11509337        7   
            310130       1726       442  679611953     533441312        7   
            455200       1400        86  998145072     490222156        3   
            571044       1098       416  831347344     750943270        4   
            658552       2183      1446  396066037     335531561        5   
            671985       1212       890  396066037     520569701        6   
            759150       1485         0  396227224     537938497        1   
            836635        108        93  732697347      23009995        5   
            928389       2364       700  396066037     520569701        5   

                        color_id  position_max  position_min  category_id  \
date_number product_id                                                      
71          151926             1          22.0           5.0            2   
            213413             1         139.0          57.0            2   
            310130             1          46.0          46.0            1   
            455200             1          53.0          36.0            2   
            571044             2         205.0         154.0            2   
            658552             1         127.0          77.0            2   
            671985             1         202.0         136.0            2   
            759150             1           NaN           NaN            0   
            836635             2           NaN           NaN            0   
            928389             1          46.0          11.0            2   

                        price  sales  
date_number product_id                
71          151926      25.95     -1  
            213413      19.95     -1  
            310130      12.95     -1  
            455200      29.95     -1  
            571044      15.95     -1  
            658552      25.95     -1  
            671985      29.95     -1  
            759150      17.95     -1  
            836635      39.95     -1  
            928389      29.95     -1

In [209]:
train.to_csv('../data/prepro_train.csv')
test.to_csv('../data/prepro_test.csv')